# Výroba a nasazení ONNX modelů  
Ve vedlejším notebooku **machine_learning** jsme se věnovali vytváření scikit-learnovských modelů. Jen okrajově jsme ale zmínili, jak vlastně natrénovaný model posléze používat. Nejjednodušší možností je jeho uložení s pomocí balíčku pickle (resp. joblib) a následné nahrání ať již obyčejným skriptem, anebo flaskovou webovou servisou. Nicméně co dělat, když vyvstane nutnost model nativně použít v javovské webaplikaci? Tehdy nezbývá než převést scikit-learnovský model do nějakého obecnějšího formátu, se kterým se dá pracovat ve více programovacích jazycích. Takovým formátem může být ONNX alias Open Neural Network Exchange. Právě otázce, jak zkonvertovat scikit-learnovký model na ONNX model a jak s takovýmto modelem pracovat jak v Pythonu, tak v Javě, se budě věnovat naše dnešní povídání.  
Aby nedošlo k nedorozumění - i když ta dvě písmena N v ONNX znamenají Neural Network, neuronové sítě zde dnes řešit nebudeme. Ne že by u nich tento problém neexistoval - koneckonců základní pytorchí ukládání modelů stojí na picklu (viz [zde](https://pytorch.org/tutorials/beginner/saving_loading_models.html)). Tensorflow pravda tohle asi tolik netrápí, neboť pro ukládání modelů používá na jazyku nezávislý formát [HFD5](https://www.tensorflow.org/tutorials/keras/save_and_load) a koneckonců má i svou [javovskou mutaci](https://www.tensorflow.org/install/lang_java?hl=en). Každopádně modely vyrobené jak Pytorchem, tak Tensorflowem lze na ONNX také převádět. Nicméně jelikož nepočítám s tím, že bych v nejbližší budoucnosti běh neuronových sítí v Javě musel řešit, nemá ani smysl, abych zde k tomu něco psal.  

## Jednoduchý model - vytvoření  v Pythonu
Zkusme pro začátek do ONNX převést jeden z nejprovařenějších modelů - klasifikaci kosatců. Zdůrazněme, že zde existují čtyři prediktory stejného datového typu, což - jak se později ukáže - situaci zjednodušuje.  
Nejprve začneme importem potřebných balíčků. Krom obvyklých podezřelých se nám zde objevují dvě nová jména - **onnxruntime** a **skl2onnx**. Jak už název napovídá, skl2onnx převádí scikit-learnový model na ONNX model. Balíček onnxruntime pak bude sloužit k práci s takto vzniklým modelem.  

In [2]:
import pandas as pd
import pickle
import skl2onnx
import onnxruntime

from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, RobustScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

pd.set_option("display.max_columns", None)  

Kosatcová data vezmeme z scikit-learnu:

In [9]:
iris_data_heap = load_iris()
iris_dataframe = pd.DataFrame(iris_data_heap["data"], columns=iris_data_heap["feature_names"])
iris_dataframe["target"] = pd.Series(iris_data_heap["target"])
iris_dataframe.sample(3, random_state=42)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
73,6.1,2.8,4.7,1.2,1
18,5.7,3.8,1.7,0.3,0
118,7.7,2.6,6.9,2.3,2


Aby ale situace nebyla od reality odtržená až tolik, náš model bude fakticky pipelina složená ze tří kroků včetně u těchto dat fakticky nepotřebného imputování.

In [11]:
final_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    MinMaxScaler(),
    LogisticRegression(max_iter=1000)
)
final_pipeline.fit(
    iris_dataframe.drop("target", axis=1), 
    iris_dataframe["target"]
)

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('minmaxscaler', MinMaxScaler()),
                ('logisticregression', LogisticRegression(max_iter=1000))])

Jelikož by mohlo být zajímavé srovnat složitost práce s obyčejným picklem a ONNXem, uložme model v obou formátech.  
U picklu je tato operace triviální:

In [13]:
with open("iris_pickle.pkl", "wb") as model_file:
    pickle.dump(final_pipeline, model_file)

U ONNX musíme před uložením použít konverzi. Konverzní funkce **skl2onnx.convert_sklearn** kromě původního modelu potřebuje i datový typ prediktorů. Přesněji potřebuje dvojici jméno vstupu a vstupní tenzor o specifikovaném datovém typu a rozměrech. Konkrétně tedy u rozměru má smysl uvést (coby druhý element listu) počet prediktorů.   

In [26]:
number_of_features = 4
initial_type = [
    ("float_input", skl2onnx.common.data_types.FloatTensorType([None, number_of_features]))
]
converted_model = skl2onnx.convert_sklearn(final_pipeline, initial_types=initial_type)
with open("iris_onnx.onnx", "wb") as model_file:
    model_file.write(converted_model.SerializeToString())

Načtěme si nyní zapicklovaný model a zkusme ho použít.

In [15]:
with open("iris_pickle.pkl", "rb") as model_file:
    loaded_iris_pickle_model = pickle.load(model_file)

Nejprve na jeden záznam...

In [18]:
one_flower = np.array([[6.1, 2.8, 4.7, 1.2]])

one_flower_pickle_pred = loaded_iris_pickle_model.predict(one_flower)
print(f"Pickled model prediction for one flower: {one_flower_pickle_pred}")

Pickled model prediction for one flower: [1]


Pak na několik záznamů najednou...

In [21]:
several_flowers = iris_dataframe.sample(3, random_state=42).drop("target", axis=1)

several_flowers_pickle_pred = loaded_iris_pickle_model.predict(several_flowers)
print(f"Pickled model prediction for several flowers: {several_flowers_pickle_pred}")

Pickled model prediction for several flowers: [1 0 2]


Podívejme se na pravděpodobnosti:

In [24]:
several_flowers_pickle_prob = loaded_iris_pickle_model.predict_proba(several_flowers)
print(f"Pickled model probability prediction for several flowers:\n{several_flowers_pickle_prob}")

Pickled model probability prediction for several flowers:
[[8.73950214e-02 6.29196011e-01 2.83408968e-01]
 [8.73572675e-01 1.19048866e-01 7.37845927e-03]
 [6.53072296e-04 1.23856107e-01 8.75490820e-01]]


To samé nyní provedeme pro ONNX model. Nejprve ho načteme.

In [27]:
loaded_iris_onnx_model = onnxruntime.InferenceSession("iris_onnx.onnx")

Pro predikování použijeme metodu **run**. Té musíme podhodit jednak jméno výstupu (lze získat pomocí metody **get_outputs**), jednak jméno vstupu (lze získat skrze metodu **get_inputs**) a v neposlední řadě vstupní data zkonvertovaná do očekávaného formátu vstupu.

In [32]:
input_name = loaded_iris_onnx_model.get_inputs()[0].name
output_name = loaded_iris_onnx_model.get_outputs()[0].name
print(f"input_name = {input_name}, output_name = {output_name}")

one_flower_onnx_pred = loaded_iris_onnx_model.run(
    [output_name], 
    {input_name: one_flower.astype(np.float32)}
)[0]
print(f"ONNX model prediction for one flower: {one_flower_onnx_pred}")

input_name = float_input, output_name = output_label
ONNX model prediction for one flower: [1]


Predikce více záznamů vypadá úplně stejně. Jen tedy musíme napřed zkonvertovat pandí dataframe na numpoidní pole.

In [39]:
several_flowers_numpy = several_flowers.to_numpy()

several_flowers_onnx_pred = loaded_iris_onnx_model.run(
    [output_name], 
    {input_name: several_flowers_numpy.astype(np.float32)}
)[0]
print(f"ONNX model prediction for several flowers: {several_flowers_onnx_pred}")

ONNX model prediction for several flowers: [1 0 2]


Pro získání pravděpodobnosti musíme použít druhou část dvousložkové návratové hodnoty metody get_outputs. Jinak ale kód vypadá stejně jako předtím.

In [42]:
proba_name = loaded_iris_onnx_model.get_outputs()[1].name
print(f"proba_name = {proba_name}")

several_flowers_onnx_pred = loaded_iris_onnx_model.run(
    [prob_name], 
    {input_name: several_flowers_numpy.astype(np.float32)}
)[0]
print(f"ONNX model probability prediction for several flowers:\n{several_flowers_onnx_pred}")

proba_name = output_probability
ONNX model probability prediction for several flowers:
[{0: 0.08739500492811203, 1: 0.6291959881782532, 2: 0.2834089994430542}, {0: 0.8735726475715637, 1: 0.11904889345169067, 2: 0.0073784636333584785}, {0: 0.0006530721439048648, 1: 0.12385611236095428, 2: 0.8754908442497253}]


## Jednoduchý model - použití v Javě

Provolávání ONNX modelu v Pythonu je sice hezké, nicméně cílem je mít možnost ho používat v rámci javovského kódu. Ten samozřejmě v Jupyteru odpálit nemůžeme - proto by se měly na Githubu vedle tohoto spisku nacházet odpovídající java soubory. No a jelikož tento typ problémů člověk obvykle řeší v pracovním softwarovém prostředí, které nemusí vždy úplně zářit novotou, bude kód kompatibilní s Javou 1.8.     
Prerekvizitou pro práci s ONNX je odpovídající knihovna. Tu si k sobě natáhneme pomocí Mavenu. To v době psaní těchto řádek (březen 2021) znamená přidat do pom.xml následující řádky:
```
<dependency>
  <groupId>com.microsoft.onnxruntime</groupId>
  <artifactId>onnxruntime</artifactId>
  <version>1.7.0</version>
</dependency>
```
Nicméně až toto budete číst, tak se raději podívejte [sem](https://search.maven.org/artifact/com.microsoft.onnxruntime/onnxruntime), zda se v mezičase neobjevily nové verze.  
Kód pro "jednokosatcový" problém vypadá následovně:
```java
package onnx_test;

import ai.onnxruntime.OnnxTensor;
import ai.onnxruntime.OrtEnvironment;
import ai.onnxruntime.OrtSession;
import ai.onnxruntime.OrtUtil;

import java.util.HashMap;

public class iris_one_flower {

    public static void main(String[] args){
        try{
            OrtEnvironment environment = OrtEnvironment.getEnvironment();
            OrtSession session = environment.createSession(
                    "c:\\vs\\programovani\\python\\workshopy\\repozitar\\machine_learning\\iris_onnx.onnx",
                    new OrtSession.SessionOptions()
            );

            float[] oneFlowerInputData = {6.1f, 2.8f, 4.7f, 1.2f};
            long[] oneFlowerInputShape = {1,4};
            Object oneFlowerReshaped = OrtUtil.reshape(oneFlowerInputData, oneFlowerInputShape);
            OnnxTensor oneFlowerTensor = OnnxTensor.createTensor(environment, oneFlowerReshaped);

            HashMap<String, OnnxTensor> inputData = new HashMap<>();
            inputData.put("float_input",oneFlowerTensor);

            OrtSession.Result results = session.run(inputData);
            System.out.println("Predicted class: " + ((long[])results.get(0).getValue())[0]);
            System.out.println("Predicted probability: " + results.get(1).getValue());
        }catch(Exception e){
            System.out.println("Following error has occurred:");
            System.out.println(e);
        }
    }
}
```
Výstupem bude
```
Predicted class: 1
Predicted probability: [{0=0.087395005, 1=0.629196, 2=0.283409}]
```
Co se tady vlastně děje? Nejprve se vytvoří environemnt objekt. To by měl být hostitelský objekt, kde může bydlet více modelů naráz. Přesněji řečeno v něm může koexistovat naráz několik session objektů, které každý obsahují právě jeden model. Sešna se vytvoří metodou *createSession*, která je nakrmena jednak cestou k modelu, jednak objektem s nastavením dané sessiony. Zde by se například dalo nastavit, že chceme použít GPU. Nicméně mi si zde vystačíme s defaultním nastavením, kdy veškerou práci obstará CPU.  
Následně je potřeba vyrobit vstupní data, resp. je přetransformovat do vhodné podoby. Nejprve hodnoty prediktorů umístíme do pole floatů. Opravdu se musí jednat o floaty a nikoli o defaultní javovský desetinnočíselný formát double, se kterým by si ONNX neporadil. Do pole longů se zase musí umístit tvar vstupních dat - v našem případě se jedná o jeden řádek se čtyřmi sloupci. Data se reshapují a zkonvertují do podoby tenzoru. Následně se umístí do mapy.  
Predikce je podobně jako v Pythonu realizovaná metodou *run*. V případě predikování třidy se nesmí zapomenout na konverzi výsledku, u pravděpodobnosti tento krok potřeba není.  
Kód zpracovávající více záznamů najednou se od předchozího příliš neliší. Pouze v poli zastupující tvar vstupu bude na prvním místě namísto jedničky trojka (na vstupu máme tři záznamy) a polem s výsledkem klasifikace musíme iterovat.
```java
package onnx_test;

import ai.onnxruntime.OnnxTensor;
import ai.onnxruntime.OrtEnvironment;
import ai.onnxruntime.OrtSession;
import ai.onnxruntime.OrtUtil;

import java.util.HashMap;

public class iris_more_flowers {
    public static void main (String[] args){
        try{
            OrtEnvironment environment = OrtEnvironment.getEnvironment();
            OrtSession session = environment.createSession(
                    "c:\\vs\\programovani\\python\\workshopy\\repozitar\\machine_learning\\iris_onnx.onnx",
                    new OrtSession.SessionOptions()
            );

            float[] moreFlowersInputData = {
                    6.1f, 2.8f, 4.7f, 1.2f,
                    5.7f, 3.8f, 1.7f, 0.3f,
                    7.7f, 2.6f, 6.9f, 2.3f
            };
            long[] moreFlowersInputShape = {3,4};
            Object moreFlowersReshaped = OrtUtil.reshape(moreFlowersInputData, moreFlowersInputShape);
            OnnxTensor moreFlowersTensor = OnnxTensor.createTensor(environment, moreFlowersReshaped);

            HashMap<String, OnnxTensor> inputData = new HashMap<>();
            inputData.put("float_input",moreFlowersTensor);

            OrtSession.Result results = session.run(inputData);
            long[] resultsArray = ((long[])results.get(0).getValue());
            for (long oneResult:resultsArray){
                System.out.println("Predicted class: " + oneResult);
            }
            System.out.println("Predicted probabilities: " + results.get(1).getValue());
        }catch(Exception e){
            System.out.println("Following error has occurred:");
            System.out.println(e);
        }
    }
}
```
V konzoli uvidíme následující:
```
Predicted class: 1
Predicted class: 0
Predicted class: 2
Predicted probabilities: [{0=0.087395005, 1=0.629196, 2=0.283409}, {0=0.87357265, 1=0.11904889, 2=0.0073784636}, {0=6.5307214E-4, 1=0.12385611, 2=0.87549084}]
```

## Složitější model - vytvoření v Pythonu
Představme si, že máme úlohu vyžadující model složitější? Co se zde ale vyšší složitostí myslí. Sice přejdeme od logistické regrese k regresnímu lesu, ale to nás netrápí. Složitost zde vnese komplikovanější pipelina a skutečnost, že v datech budeme mít prediktory více typů.  
Použijeme kagglovská data z úlohy na [ceny nemovitostí](https://www.kaggle.com/c/house-prices-advanced-regression-techniques). Nicméně pozorujeme, že prediktorů je pro naše účel snad až moc - omezíme se proto jen na první čtyři - MSSubClass, MSZoning, LotFrontage a LotArea. Přesnost modelu sice půjde pod kytičky, nicméně o to nám zde ani nejde - cílem je vytvořit ONNX model a porovnat jeho výstupy s modelem normálním.

In [8]:
wanted_columns = ["MSSubClass", "MSZoning", "LotFrontage", "LotArea", "SalePrice"]
houses_dataset = pd.read_csv("train.csv", header=0, sep=",")[wanted_columns]
houses_dataset.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,SalePrice
0,60,RL,65.0,8450,208500
1,20,RL,80.0,9600,181500
2,60,RL,68.0,11250,223500
3,70,RL,60.0,9550,140000
4,60,RL,84.0,14260,250000


Podívejme se na datové typy prediktorů.

In [9]:
houses_dataset.dtypes

MSSubClass       int64
MSZoning        object
LotFrontage    float64
LotArea          int64
SalePrice        int64
dtype: object

Z pohledu ONNX by se hodilo, kdyby všechny one-hot encodované sloupce měly stejný typ a kdyby všechny numerické sloupce byly floaty.

In [10]:
houses_dataset["MSSubClass"] = houses_dataset["MSSubClass"].astype(str)
houses_dataset["LotArea"] = houses_dataset["LotArea"].astype(float)
houses_dataset.dtypes

MSSubClass      object
MSZoning        object
LotFrontage    float64
LotArea        float64
SalePrice        int64
dtype: object

Vytvořme si pipelinu. Zde začnou nastávat problémy. Ne každá věc v scikit-learnu je převeditelná do ONNXu. Například použití FunctionTransformeru vede k vzniku chyby 
```
RuntimeError: FunctionTransformer is not supported unless the transform function is None (= identity). You may raise an issue at https://github.com/onnx/sklearn-onnx/issues.
```
Věc se má totiž tak, že by se skrze tento transformer mohla dál dostat funkce, se kterou si ONNX nedokáže [poradit](https://github.com/onnx/sklearn-onnx/issues/278). V našem konkrétním případě jsme přišli o logaritmování. To by v běžném provozu znamenalo, že logaritmování bychom museli při trénování v Pythonu i predikci v Javě provést ručně. Jelikož v tomto příkladu na kvalitě predikce nebazírujeme, necháme to být. Další věcí, která bude muset zmizet, je imputování pro stringy (viz [zde](http://onnx.ai/sklearn-onnx/auto_examples/plot_complex_pipeline.html#example-complex-pipeline)).

In [11]:
col_one_hot = ["MSSubClass", "MSZoning"]
col_scaler = ["LotFrontage", "LotArea"]

one_hot_pipeline = make_pipeline(
    #SimpleImputer(strategy="constant", fill_value="NotPresent"),
    OneHotEncoder(handle_unknown="ignore")
)

scaler_pipeline = make_pipeline(
    SimpleImputer(strategy="constant", fill_value=0),
    #FunctionTransformer(func=np.log1p),
    RobustScaler()
)

joined_pipeline = ColumnTransformer([
    ("scaler", scaler_pipeline, col_scaler),
    ("one_hot", one_hot_pipeline, col_one_hot)
], remainder="drop")

final_pipeline_forest = make_pipeline(
    joined_pipeline,
    RandomForestRegressor()
)

final_pipeline_forest.fit(
    houses_dataset.drop("SalePrice", axis=1), 
    houses_dataset["SalePrice"]
)
final_pipeline_forest

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('scaler',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['LotFrontage', 'LotArea']),
                                                 ('one_hot',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['MSSubClass',
                                                   

Aplikujme scikit-learnovský model na několik záznamů...

In [31]:
more_houses = houses_dataset.sample(3, random_state=42).drop("SalePrice", axis=1)
more_houses

,MSSubClass,MSZoning,LotFrontage,LotArea
892,20,RL,70.0,8414.0
1105,60,RL,98.0,12256.0
413,30,RM,56.0,8960.0


In [21]:
final_pipeline_forest.predict(more_houses)

array([153684.83333333, 331668.93      , 106657.        ])

V kontrastu ke kosatcům musíme v initial_type vyrobit pro každý prediktor separátní tuple o jednom sloupci

In [22]:
initial_type = [
    ("MSSubClass", skl2onnx.common.data_types.StringTensorType([None, 1])),
    ("MSZoning", skl2onnx.common.data_types.StringTensorType([None, 1])),  
    ("LotFrontage", skl2onnx.common.data_types.FloatTensorType([None, 1])),
    ("LotArea", skl2onnx.common.data_types.FloatTensorType([None, 1]))

]
converted_model = skl2onnx.convert_sklearn(final_pipeline_forest, initial_types=initial_type)
with open("houses_onnx.onnx", "wb") as model_file:
    model_file.write(converted_model.SerializeToString())

Zpracujme data k ohodnocení do podoby, ve které jim bude na chvíli načtený ONNX model rozumět. Tj. hodnoty číselných sloupců převedeme na float a všechny sloupce reshapujeme:

In [26]:
inputs = {col_name: more_houses[col_name].values for col_name in more_houses.columns}
for col_name in ["LotFrontage", "LotArea"]:
    inputs[col_name] = inputs[col_name].astype(np.float32)
for col_name in inputs:
    inputs[col_name] = inputs[col_name].reshape((inputs[col_name].shape[0], 1))
inputs

{'MSSubClass': array([['20'],
        ['60'],
        ['30']], dtype=object),
 'MSZoning': array([['RL'],
        ['RL'],
        ['RM']], dtype=object),
 'LotFrontage': array([[70.],
        [98.],
        [56.]], dtype=float32),
 'LotArea': array([[ 8414.],
        [12256.],
        [ 8960.]], dtype=float32)}

Predikci v Pythonu pak realizujeme následujícím způsobem:

In [30]:
loaded_houses_onnx_model = onnxruntime.InferenceSession("houses_onnx.onnx")
predicted_prices = loaded_houses_onnx_model.run(
    None, inputs
)[0]
print(f"Predicted prices:\n{predicted_prices}")

Predicted prices:
[[153684.83]
 [330689.47]
 [106657.  ]]


## Složitější model - použití v Javě  
Javovský kód pro složitější model vypadá následovně:
```java
package onnx_test;

import ai.onnxruntime.OnnxTensor;
import ai.onnxruntime.OrtEnvironment;
import ai.onnxruntime.OrtSession;
import ai.onnxruntime.OrtUtil;

import java.util.HashMap;

public class houses_more_houses {
    public static void main(String[] args){
        try{
            OrtEnvironment environment = OrtEnvironment.getEnvironment();
            OrtSession session = environment.createSession(
                    "c:\\vs\\programovani\\python\\workshopy\\repozitar\\machine_learning\\houses_onnx.onnx",
                    new OrtSession.SessionOptions()
            );

            HashMap<String, OnnxTensor> inputData = new HashMap<>();
            long[] housesInputShape = {3,1};

            String[] dataMSSubClass = {"20", "60", "30"};
            OnnxTensor tensorMSSubClassData = OnnxTensor.createTensor(
                    environment, dataMSSubClass, housesInputShape
            );
            inputData.put("MSSubClass",tensorMSSubClassData);

            String[] dataMSZoningData = {"RL", "RL", "RM"};
            OnnxTensor tensorMSZoning = OnnxTensor.createTensor(
                    environment, dataMSZoningData, housesInputShape
            );
            inputData.put("MSZoning",tensorMSZoning);

            float[] dataLotFrontage = {70.0f, 98.0f, 56.0f};
            Object reshapedLotFrontage = OrtUtil.reshape(dataLotFrontage, housesInputShape);
            OnnxTensor tensorLotFrontage = OnnxTensor.createTensor(environment, reshapedLotFrontage);
            inputData.put("LotFrontage",tensorLotFrontage);

            float[] dataLotArea = {8414.0f, 12256.0f, 8960.0f};
            Object reshapedLotArea = OrtUtil.reshape(dataLotArea, housesInputShape);
            OnnxTensor tensorLotArea = OnnxTensor.createTensor(environment, reshapedLotArea);
            inputData.put("LotArea",tensorLotArea);

            OrtSession.Result results = session.run(inputData);
            float[][] resultsArray = (float[][])results.get(0).getValue();
            for (float[] oneResult:resultsArray){
                System.out.println(oneResult[0]);
            }
        }catch (Exception e){
            System.out.println("Following error has occurred:");
            System.out.println(e);
        }
    }
}
```
Výstupem bude
```
153684.83
330689.47
106657.0
```
Čím se liší od prvotního javovského kódu? Nyní má každý prediktor svůj vlastní tenzor a svůj vlastní záznam v mapě inputData. Za zmínku stojí skutečnost, že u stringů neprovádíme reshapování. Další rozdíl spočívá v tom, že se výsledek nepřevádí na jednorozměrné, nýbrž na dvojrozměrné pole. Nicméně ve výsledku se koukáme na praktický identický kód.